In [2]:
# library
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchmetrics import R2Score, MeanSquaredError

r2score = R2Score()
msescore = MeanSquaredError()

torch.manual_seed(2)
np.random.seed(2)
torch.set_printoptions(precision=8)

In [3]:
# Model (system)
class System(torch.nn.Module):
    def __init__(self):
        super(System, self).__init__()
        self.input_ini    = torch.nn.Linear(1, 4)
        self.input_signal = torch.nn.Linear(4, 4)
        self.input_t      = torch.nn.Linear(1, 4)

    def forward(self, x_0, u, t):
        z_ini   = torch.selu(self.input_ini(x_0))
        z_input = torch.selu(self.input_signal(u))
        z_t     = torch.selu(self.input_t(t))
        z       = z_ini * z_input * z_t
        z       = torch.sum(z, dim=1).reshape(-1,1)
        return z

In [4]:
# Model error
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x0_data, testset.u_data, testset.t_data)

    r2  = r2score(pred_Y, testset.y_data)
    mse = msescore(pred_Y, testset.y_data)
    return r2.item(), mse.item()

In [5]:
# Data
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start, stop):
    df = pd.read_csv(src_file)
    X0 = np.array(df['X0'][start:stop]).reshape(-1,1)
    U1 = np.array(df['U1'][start:stop])
    U2 = np.array(df['U2'][start:stop])
    U3 = np.array(df['U3'][start:stop])
    U4 = np.array(df['U4'][start:stop])
    U  = np.column_stack((U1, U2, U3, U4))
    T  = np.array(df['T'][start:stop]).reshape(-1,1)
    Y  = np.array(df['Y'][start:stop]).reshape(-1,1)

    self.x0_data = torch.tensor(X0, dtype=torch.float32)
    self.u_data  = torch.tensor(U,  dtype=torch.float32)
    self.t_data  = torch.tensor(T,  dtype=torch.float32)
    self.y_data  = torch.tensor(Y,  dtype=torch.float32)

  def __len__(self):
    return len(self.x0_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    x0  = self.x0_data[idx]
    u   = self.u_data[idx]
    t   = self.t_data[idx]
    y   = self.y_data[idx]
    sample = {'x0':x0, 'u':u, 't':t, 'y':y}
    return sample

In [6]:
# Early stopping
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.0001*np.max(list[0: -patience])):
            return 1
    return 0

In [7]:
# Train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)

    R2  = np.array([])
    MSE = np.array([])
    for epoch in range(0, max_epochs+1):
        net.train()
        loss  = 0
        count = 0
        for (_, batch) in enumerate(train_ldr):
            X0 = batch['x0']
            U  = batch['u']
            T  = batch['t']
            Y  = batch['y']

            optimizer.zero_grad()
            output = net(X0, U, T)             # compute the output of the Network
            loss_val = loss_func(output, Y)    # loss function
            loss += loss_val.item()            # accumulate
            loss_val.backward()                # gradients
            optimizer.step()                   # update paramters
            count += 1
        
        net.eval()
        R2  = np.append(R2, eval(net, test_ds)[0])
        MSE = np.append(MSE, eval(net, test_ds)[1])

        if(epoch%500==0):
            print("epoch = %5d \t loss = %12.4f \t R2 = %12.4f \t MSE = %12.4f" % (epoch, loss/count, eval(net, test_ds)[0], eval(net, test_ds)[1]))
        
        if(early_stop(list = R2, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2, MSE

In [8]:
# Create Dataset and DataLoader objects
src_file = 'training_data.csv'
train_ds = Data(src_file, 0, 4096)
test_ds  = Data(src_file, 4096, 7980)

# Create network
device = torch.device("cpu")
net = System().to(device)

# train model
lr         = 0.001
min_epochs = 500
max_epochs = 100000
patience   = 300
R2, MSE = train(net, train_ds, test_ds, lr, min_epochs, max_epochs, patience)

epoch =     0 	 loss =   22048.4746 	 R2 =   -4122.8574 	 MSE =   25656.3125
epoch =   500 	 loss =     111.7399 	 R2 =     -22.2664 	 MSE =     144.7503
epoch =  1000 	 loss =      66.5505 	 R2 =     -13.9911 	 MSE =      93.2662
epoch =  1500 	 loss =      46.1349 	 R2 =      -8.4572 	 MSE =      58.8373
epoch =  2000 	 loss =      32.4966 	 R2 =      -4.8785 	 MSE =      36.5730
epoch =  2500 	 loss =      23.3050 	 R2 =      -2.7568 	 MSE =      23.3725
epoch =  3000 	 loss =      17.0519 	 R2 =      -1.5283 	 MSE =      15.7296
epoch =  3500 	 loss =      12.4642 	 R2 =      -0.7398 	 MSE =      10.8239
epoch =  4000 	 loss =       8.8966 	 R2 =      -0.1805 	 MSE =       7.3445
epoch =  4500 	 loss =       6.1503 	 R2 =       0.2144 	 MSE =       4.8876
epoch =  5000 	 loss =       4.0910 	 R2 =       0.4816 	 MSE =       3.2255
epoch =  5500 	 loss =       2.4067 	 R2 =       0.6425 	 MSE =       2.2240
epoch =  6000 	 loss =       1.3343 	 R2 =       0.7179 	 MSE =       1.7550

In [9]:
torch.save(net, 'system.pt')